In [1]:
inp="""$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [76]:
class Dir:
    def __init__(self, name):
        self.name = name
        self.content = {}
    def add(self, o):
        self.content[o.name] = o
        o.setParent(self)
    def __repr__(self) -> str:
        return f"{self.name} (dir)"
    def size(self) -> int:
        return sum(o.size() for o in self.content.values())
    def setParent(self, d):
        self.parent = d
    def cd(self, target):
        if target == "..":
            return self.parent
        if type(self.content[target]) is not Dir:
            raise ValueError("not a dir")
        return self.content[target]
    def collect(self, dirs={}) -> int:
        total = 0
        if self in dirs:
            return dirs[self]
        for o in self.content.values():
            total += o.collect(dirs)
        dirs[self] = total
        return total
            
            



class File:
    def __init__(self, name, size):
        self.name = name
        self._size = size
    def __repr__(self) -> str:
        return f"{self.name} (file, size={self._size})"
    def size(self) -> int:
        return self._size
    def collect(self, dirs) -> int:
        return self.size()
    def setParent(self, d: Dir):
        self.parent = d
        
def pretty(o, indent):
    return " "*indent + "- " + str(o)

d = Dir("/")
d2= Dir("bla")
d2.add(File("foo", 123))
d.add(d2)
d.add(d2)
d.size()

123

In [77]:
cur = d.cd("bla").cd("..")
cur.collect()

123

In [81]:
from collections import deque

def parse(inp:str):
    root = Dir("/")
    cur: Dir

    i = 0
    lines = inp.split("\n")
    while i < len(lines):
        #print(lines[i])

        while i < len(lines) and not lines[i].startswith("$"):
            x, name = lines[i].split(" ")
            i+=1
            #print(name)
            if x == "dir":
                #print("adding dir")
                cur.add(Dir(name))
                #print(cur)
                #print(cur.content)
            else:
                cur.add(File(name, int(x)))

        if i >= len(lines):
            break

        line = lines[i]
        if line == "$ cd /":
            cur = root
        elif line.startswith("$ cd"):
            target = line.removeprefix("$ cd ")
            cur = cur.cd(target)
        
        i+=1
        
    return root
parse(inp)

/ (dir)

In [82]:
def part_1(raw: str) -> int:
    root = parse(raw)
    sizes = {}
    root.collect(sizes)
    return sum(size for size in sizes.values() if size <= 100000)



part_1(inp)

95437

In [83]:
data = open("data/07.txt").read()

In [84]:
part_1(data)

1517599

In [87]:
space = 70000000
needed = 30000000

def part_2(raw):
    root = parse(raw)
    sizes = {}
    root.collect(sizes)

    unused = space - sizes[root]
    to_free = needed - unused
    cand = root
    for dir, size in sizes.items():
        if size < to_free:
            continue
        if size > sizes[cand]:
            continue
        cand = dir
    return cand, sizes[cand]
        


In [89]:
part_2(data)

(rbh (dir), 2481982)